In [68]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt  
import yfinance as yf 
from datetime import datetime, timedelta

In [69]:
class FinancialData():

    def __init__(self, tickers_csv_path, num_tickers):
        self.tickers_csv_path = tickers_csv_path
        self.num_tickers = num_tickers
        self.tickers = self.extract_tickers()
        self.data_clean = None
        self.mu = None
        self.Epsilon = None
        
    def extract_tickers(self):
        tickers_df = pd.read_csv(self.tickers_csv_path)
        list_tickers = tickers_df.loc[:self.num_tickers - 1, 'Ticker'].values.tolist()
        return list_tickers
    
    def get_returns(self, start, end, period="1d"):
        end = start
        start = end
        data = yf.download(self.tickers, start=start, end=end, period=period)['Adj Close']
        data = data.pct_change(fill_method=None).dropna()
        data.index = data.index.strftime("%Y-%m-%d")
        self.data_clean = data

    def calculate_statistics(self):
        if self.data_clean is not None:
            self.mu = self.data_clean.mean(axis=1).values
            self.Epsilon = self.data_clean.cov().to_numpy()
        else:
            print("get returns before please")


In [70]:
# input 

tickers_csv_path = "C:\\Users\\Nicola\\clones_github\\DL-Portfolio-CMAP\\data\\S&P 500 tickers.csv"
num_tickers = 5 
end = datetime.today()
start = end - timedelta(days=4)

In [71]:
# output

data_extractor_5 = FinancialData(tickers_csv_path, num_tickers)
data_extractor_5.extract_tickers()
data_extractor_5.get_returns(start=start, end=end)
data_extractor_5.calculate_statistics()
rets_df = data_extractor_5.data_clean
mu_5 = data_extractor_5.mu
Eps_5 = data_extractor_5.Epsilon

[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['ABBV', 'AOS', 'ACN', 'ABT', 'MMM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-10-11 22:48:36.324863 -> 2024-10-11 22:48:36.324863)')
c:\Users\Nicola\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Nicola\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\Nicola\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\Nicola\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fac